In [1]:
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.express as px

from Almanac.Data import get_weather_data
from Almanac.Data import get_frost_dates
from Almanac.Data import binarize
from Almanac.Models import hw_weekly_frost_date_forecast
from Almanac.Models import sarima_forecast
from datetime import datetime, timedelta
import time

In [2]:
app = Dash(__name__)

In [3]:
app.layout = html.Div(
    [
        "Location:",
        html.Br(),
        dcc.Input(
            id="location",
            type="text",
            placeholder="Enter Location",
            debounce=True,
        ),
        html.Div(id="text-output"),
        html.Br(),
        # html.Div(id="hidden location",style={"display":"none"}),
        "Start Date:",
        html.Br(),
        dcc.Input(
            id="start-date",
            type="text",
            placeholder="yyyy-mm-dd",
            debounce=True,
        ),
        html.Div(id="start-date-output"),
        html.Br(),
        "End Date:",
        html.Br(),
        dcc.Input(
            id="end-date", type="text", placeholder="yyyy-mm-dd", debounce=True
        ),
        html.Div(id="end-date-output"),
        html.Br(),
        "Model:",
        html.Br(),
        dcc.Dropdown(
            ["Holt Winters", "SARIMA"],
            "Holt Winters",
            id="model dropdown",
        ),
        html.Br(),
        dcc.Graph(id="fig1"),
    ]
)

In [4]:
@app.callback(Output("text-output", "children"), Input("location", "value"))
def update_location_div(location):
    if not location:
        raise PreventUpdate
    return f"Location Selected: {location}"

In [5]:
@app.callback(
    Output("start-date-output", "children"), Input("start-date", "value")
)
def update_start_div(start):
    if not start:
        raise PreventUpdate
    return f"Start date Selected: {start}"

In [6]:
@app.callback(
    Output("end-date-output", "children"), Input("end-date", "value")
)
def update_end_div(end):
    if not end:
        raise PreventUpdate
    return f"End date Selected: {end}"

In [7]:
# @app.callback(
# Output("hidden location","data"),
# Input("location","value"),
# Input("start-date","value"),
# Input("end-date","value")
# )
# def get_data(location,start,end):
#     if (not location) or (not start) or (not end):
#         raise PreventUpdate

#     data = get_weather_data(location,start,end)
#     return data

In [8]:
@app.callback(
    Output("fig1", "figure"),
    Input("location", "value"),
    Input("start-date", "value"),
    Input("end-date", "value"),
    Input("model dropdown", "value"),
    # Input("hidden location","data")
)
def generate_figure(location, start, end, model):
    #     print(f"{location}, {start}, {end}")
    if not end:
        raise PreventUpdate

    for i in range(3):
        try:
            data = get_weather_data(location, start, end)
        except Exception:
            print(f"Fetching Weather Data attempt {i+1}")
            time.sleep(3)
            continue

    if model == "Holt Winters":
        prediction, os = hw_weekly_frost_date_forecast(data)
        figure = px.line(x=prediction.index, y=prediction)
        figure.update_layout(
            title={
                "text": (
                    f"Holt Winters Prediction Weekly Min Temperature {figure.data[0].x[0].strftime('%Y-%m-%d')}"
                    f" to {figure.data[0].x[-1].strftime('%Y-%m-%d')} for {location}"
                ),
                "x": 0.5,
                "y": 0.95,
                "xanchor": "center",
            },
            title_font={"color": "deepskyblue", "size": 17},
        )
        figure.update_xaxes(title="Date")
        figure.update_yaxes(title="Temperature")
        return figure

    if model == "SARIMA":
        df = data["tmin"].resample("W").min()
        prediction = sarima_forecast(
            df, start=df.index[-1], end=df.index[-1] + timedelta(days=365)
        )

        figure = px.line(x=prediction.index, y=prediction)
        figure.update_layout(
            title={
                "text": (
                    f"SARIMA Prediction Weekly Min Temperature {figure.data[0].x[0].strftime('%Y-%m-%d')}"
                    f" to {figure.data[0].x[-1].strftime('%Y-%m-%d')} for {location}"
                ),
                "x": 0.5,
                "y": 0.95,
                "xanchor": "center",
            },
            title_font={"color": "deepskyblue", "size": 17},
        )
        figure.update_xaxes(title="Date")
        figure.update_yaxes(title="Temperature")
        return figure

SyntaxError: unterminated string literal (detected at line 29) (1629631550.py, line 29)

In [ ]:
app.run()

make a new callback for fetching data that looks for location, start, and end not == none. This will update a hidden html.Div that generate figure can listen for.